In [1]:
import numpy as np
import pandas as pd

Examine imports use table

In [2]:
data = pd.read_csv('data/iot.csv', thousands=',')
data.head()

,product,"Products of agriculture, hunting and related services","Products of forestry, logging and related services",Fish and other fishing products; aquaculture products; support services to fishing,Coal and lignite,Extraction Of Crude Petroleum And Natural Gas & Mining Of Metal Ores,Other mining and quarrying products,Mining support services,Preserved meat and meat products,"Processed and preserved fish, crustaceans, molluscs, fruit and vegetables",...,Human health services,Residential Care & Social Work Activities,"Creative, arts and entertainment services","Libraries, archives, museums and other cultural services",Gambling and betting services,Sports services and amusement and recreation services,Services furnished by membership organisations,Repair services of computers and personal and household goods,Other personal services,Services of households as employers of domestic personnel
0,"Products of agriculture, hunting and related s...",3198,0,0,0,0,5,0,5007,1238,...,14,32,0,0,20,2,0,2,0,0
1,"Products of forestry, logging and related serv...",114,309,0,0,0,0,0,0,0,...,0,1,0,0,0,20,0,2,0,0
2,Fish and other fishing products; aquaculture p...,0,0,35,0,0,0,0,6,847,...,0,0,0,0,0,0,0,0,0,0
3,Coal and lignite,0,0,0,12,0,4,0,0,0,...,0,0,0,0,0,0,18,0,0,0
4,Extraction Of Crude Petroleum And Natural Gas ...,0,0,0,0,729,9,0,0,0,...,0,0,0,0,0,3,0,1,0,0


In [3]:
# convert wide table to long table
data_long = pd.melt(data, 
                     id_vars='product', 
                     value_vars=data.columns.to_list()[1:]).dropna()

data_long.shape

(11025, 3)

In [4]:
# Strip whitespace from start and finish

def space_stripper(x):
    return x.strip()

data_long[['product','variable']] = data_long[['product','variable']].applymap(lambda x: space_stripper(x))

In [5]:
data_long.rename(columns={'product':'domestic input requirements'}, inplace=True)
data_long.rename(columns={'variable':'output product'}, inplace=True)
data_long['value'] = data_long['value'].astype(int)

In [6]:
data_long['proportion'] = data_long['value'] / data_long.groupby('output product')['value'].transform('sum')

data_long.to_csv('data/iot_cleaned.csv')

In [7]:
data_long.head()

,domestic input requirements,output product,value,proportion
0,"Products of agriculture, hunting and related s...","Products of agriculture, hunting and related s...",3198,0.211229
1,"Products of forestry, logging and related serv...","Products of agriculture, hunting and related s...",114,0.007530
2,Fish and other fishing products; aquaculture p...,"Products of agriculture, hunting and related s...",0,0.000000
3,Coal and lignite,"Products of agriculture, hunting and related s...",0,0.000000
4,Extraction Of Crude Petroleum And Natural Gas ...,"Products of agriculture, hunting and related s...",0,0.000000


In [9]:
iot_use = pd.read_csv('data/iot_cleaned.csv')

detail = set(list(data_long['output product']))
detail

{'Accommodation services',
 'Accounting, bookkeeping and auditing services; tax consulting services',
 'Advertising and market research services',
 'Air and spacecraft and related machinery',
 'Air transport services',
 'Alcoholic beverages  & Tobacco products',
 'Architectural and engineering services; technical testing and analysis services',
 'Bakery and farinaceous products',
 'Basic iron and steel',
 'Basic pharmaceutical products and pharmaceutical preparations',
 'Cement, lime, plaster and articles of concrete, cement and plaster',
 'Coal and lignite',
 'Coke and refined petroleum products',
 'Computer programming, consultancy and related services',
 'Computer, electronic and optical products',
 'Construction',
 'Creative, arts and entertainment services',
 'Dairy products',
 'Dyestuffs, agro-chemicals - 20.12/20',
 'Education services',
 'Electrical equipment',
 'Electricity, transmission and distribution',
 'Employment services',
 'Extraction Of Crude Petroleum And Natural Gas

In [10]:
iot_subset = iot_use[iot_use['output product']==detail]

iot_subset = iot_subset[iot_subset['proportion']>0]

In [12]:
import plotly.express as px

fig = px.pie(iot_subset, values='proportion', names='domestic input requirements')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.write_html('second_figure.html', auto_open=True)